# 多智能体模拟环境：宠物园

在这个例子中，我们展示了如何使用模拟环境定义多智能体的模拟。就像我们之前使用Gymnasium定义单智能体的例子一样，我们创建了一个智能体-环境循环，其中环境是外部定义的。主要区别在于我们现在实现了多智能体的交互循环。我们将使用Petting Zoo库，它是Gymnasium的多智能体对应库。

## 安装 `pettingzoo` 和其他依赖项

In [1]:
# 安装所需的库
!pip install pettingzoo pygame rlcard

## 导入模块

In [2]:
# 导入必要的库
import collections
import inspect

import tenacity
from langchain.output_parsers import RegexParser
from langchain.schema import (
    HumanMessage,  # 导入HumanMessage类
    SystemMessage,  # 导入SystemMessage类
)
from langchain_openai import ChatOpenAI  # 导入ChatOpenAI类

## `GymnasiumAgent`
在这里，我们复制了与[我们的Gymnasium示例](https://python.langchain.com/en/latest/use_cases/agent_simulations/gymnasium.html)中定义的相同的`GymnasiumAgent`。如果经过多次重试后仍然无法采取有效的动作，它将随机选择一个动作。

In [3]:
class GymnasiumAgent:
    @classmethod
    def get_docs(cls, env):
        return env.unwrapped.__doc__  # 获取环境的文档字符串

    def __init__(self, model, env):
        self.model = model  # 模型
        self.env = env  # 环境
        self.docs = self.get_docs(env)  # 获取环境的文档字符串

        self.instructions = """
你的目标是最大化你的回报，即你所获得的奖励的总和。
我会给你一个观察值、奖励、终止标志、截断标志和迄今为止的回报，格式如下：

观察值: <observation>
奖励: <reward>
终止标志: <termination>
截断标志: <truncation>
回报: <sum_of_rewards>

你需要用一个动作来回应，格式如下：

动作: <action>

你需要将 <action> 替换为你的实际动作。
除此之外，不要做任何其他操作，只需返回动作。
"""
        self.action_parser = RegexParser(
            regex=r"Action: (.*)", output_keys=["action"], default_output_key="action"
        )  # 动作解析器

        self.message_history = []  # 消息历史记录
        self.ret = 0  # 回报

    def random_action(self):
        action = self.env.action_space.sample()  # 随机选择一个动作
        return action

    def reset(self):
        self.message_history = [
            SystemMessage(content=self.docs),  # 添加环境的文档字符串到消息历史记录
            SystemMessage(content=self.instructions),  # 添加指令到消息历史记录
        ]

    def observe(self, obs, rew=0, term=False, trunc=False, info=None):
        self.ret += rew  # 更新回报

        obs_message = f"""
观察值: {obs}
奖励: {rew}
终止标志: {term}
截断标志: {trunc}
回报: {self.ret}
        """
        self.message_history.append(HumanMessage(content=obs_message))  # 添加观察消息到消息历史记录
        return obs_message

    def _act(self):
        act_message = self.model.invoke(self.message_history)  # 使用模型进行动作选择
        self.message_history.append(act_message)  # 添加动作消息到消息历史记录
        action = int(self.action_parser.parse(act_message.content)["action"])  # 解析动作消息中的动作
        return action

    def act(self):
        try:
            for attempt in tenacity.Retrying(
                stop=tenacity.stop_after_attempt(2),  # 最多尝试2次
                wait=tenacity.wait_none(),  # 重试之间没有等待时间
                retry=tenacity.retry_if_exception_type(ValueError),  # 如果出现 ValueError 异常则重试
                before_sleep=lambda retry_state: print(
                    f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
                ),  # 打印错误信息并重试
            ):
                with attempt:
                    action = self._act()  # 执行动作选择
        except tenacity.RetryError:
            action = self.random_action()  # 如果重试失败，则随机选择一个动作
        return action  # 返回动作

## 主循环

In [4]:
def main(agents, env):
    # 重置环境
    env.reset()

    # 对每个智能体进行重置
    for name, agent in agents.items():
        agent.reset()

    # 对环境中的每个智能体进行迭代
    for agent_name in env.agent_iter():
        # 获取环境的最新状态
        observation, reward, termination, truncation, info = env.last()
        # 让智能体观察环境并返回观察信息
        obs_message = agents[agent_name].observe(
            observation, reward, termination, truncation, info
        )
        print(obs_message)
        # 如果环境终止或被截断，则不执行动作
        if termination or truncation:
            action = None
        else:
            # 让智能体执行动作
            action = agents[agent_name].act()
        print(f"Action: {action}")
        # 在环境中执行动作
        env.step(action)
    # 关闭环境
    env.close()

## `PettingZooAgent`

`PettingZooAgent`扩展了`GymnasiumAgent`以适应多智能体环境。主要的区别包括：
- `PettingZooAgent`接受一个`name`参数，用于在多个智能体中进行标识
- `get_docs`函数的实现方式不同，因为`PettingZoo`仓库的结构与`Gymnasium`仓库的结构不同。

In [5]:
# 定义一个PettingZooAgent类，继承自GymnasiumAgent类
class PettingZooAgent(GymnasiumAgent):
    # 定义一个类方法get_docs，用于获取环境的文档信息
    @classmethod
    def get_docs(cls, env):
        return inspect.getmodule(env.unwrapped).__doc__

    # 定义初始化方法，接受name、model和env三个参数
    def __init__(self, name, model, env):
        # 调用父类的初始化方法
        super().__init__(model, env)
        # 设置实例变量name
        self.name = name

    # 定义random_action方法，用于生成随机动作
    def random_action(self):
        # 从环境的动作空间中随机采样一个动作
        action = self.env.action_space(self.name).sample()
        return action

## 石头，剪刀，布
现在我们可以使用`PettingZooAgent`来运行一个多智能体石头，剪刀，布游戏的模拟。

In [6]:
from pettingzoo.classic import rps_v2  # 导入rps_v2环境

# 创建一个rps_v2环境，最大周期为3，渲染模式为"human"
env = rps_v2.env(max_cycles=3, render_mode="human")

# 创建一个代理字典，代理名称为环境中可能的代理名称，值为PettingZooAgent对象，使用ChatOpenAI模型，温度为1
agents = {
    name: PettingZooAgent(name=name, model=ChatOpenAI(temperature=1), env=env)
    for name in env.possible_agents
}

# 调用main函数，传入代理字典和环境对象
main(agents, env)


Observation: 3
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 1

Observation: 3
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 1

Observation: 1
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 2

Observation: 1
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 1

Observation: 1
Reward: 1
Termination: False
Truncation: False
Return: 1
        
Action: 0

Observation: 2
Reward: -1
Termination: False
Truncation: False
Return: -1
        
Action: 0

Observation: 0
Reward: 0
Termination: False
Truncation: True
Return: 1
        
Action: None

Observation: 0
Reward: 0
Termination: False
Truncation: True
Return: -1
        
Action: None


## `ActionMaskAgent`

一些 `PettingZoo` 环境提供了一个 `action_mask` 来告诉代理程序哪些动作是有效的。`ActionMaskAgent` 是 `PettingZooAgent` 的子类，使用来自 `action_mask` 的信息来选择动作。

In [7]:
# 定义一个名为ActionMaskAgent的类，继承自PettingZooAgent类
class ActionMaskAgent(PettingZooAgent):
    # 初始化方法，接受name、model和env三个参数
    def __init__(self, name, model, env):
        # 调用父类的初始化方法
        super().__init__(name, model, env)
        # 创建一个长度为1的双向队列作为观察缓冲区
        self.obs_buffer = collections.deque(maxlen=1)

    # 定义一个随机动作的方法
    def random_action(self):
        # 获取观察缓冲区中的最后一个观察
        obs = self.obs_buffer[-1]
        # 从环境的动作空间中随机采样一个动作，根据观察中的“action_mask”进行采样
        action = self.env.action_space(self.name).sample(obs["action_mask"])
        return action

    # 定义重置方法
    def reset(self):
        # 初始化消息历史，包括系统消息和指令消息
        self.message_history = [
            SystemMessage(content=self.docs),
            SystemMessage(content=self.instructions),
        ]

    # 定义观察方法，接受obs、rew、term、trunc和info五个参数
    def observe(self, obs, rew=0, term=False, trunc=False, info=None):
        # 将观察添加到观察缓冲区中
        self.obs_buffer.append(obs)
        # 调用父类的观察方法
        return super().observe(obs, rew, term, trunc, info)

    # 定义私有的动作方法
    def _act(self):
        # 有效动作指令
        valid_action_instruction = "Generate a valid action given by the indices of the `action_mask` that are not 0, according to the action formatting rules."
        # 将有效动作指令添加到消息历史中
        self.message_history.append(HumanMessage(content=valid_action_instruction))
        # 调用父类的动作方法
        return super()._act()

## 井字游戏
这是一个使用`ActionMaskAgent`的井字游戏示例。

In [8]:
from pettingzoo.classic import tictactoe_v3

env = tictactoe_v3.env(render_mode="human")
agents = {
    name: ActionMaskAgent(name=name, model=ChatOpenAI(temperature=0.2), env=env)
    for name in env.possible_agents
}
main(agents, env)


Observation: {'observation': array([[[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]]], dtype=int8), 'action_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int8)}
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 0
     |     |     
  X  |  -  |  -  
_____|_____|_____
     |     |     
  -  |  -  |  -  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     

Observation: {'observation': array([[[0, 1],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0]]], dtype=int8), 'action_mask': array([0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int8)}
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 1
     |     |     
  X  |  -  |  -  
_____|_____|_____
     |     |     
  O  |  -  |  -  
_____|_____|_____
     |     |     
  -  |  -  |  -  
     |     |     

Observation

## 德州扑克无限制版
这是一个使用`ActionMaskAgent`的德州扑克无限制版游戏的示例。

In [9]:
from pettingzoo.classic import texas_holdem_no_limit_v6

env = texas_holdem_no_limit_v6.env(num_players=4, render_mode="human")
agents = {
    name: ActionMaskAgent(name=name, model=ChatOpenAI(temperature=0.2), env=env)
    for name in env.possible_agents
}
main(agents, env)


Observation: {'observation': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 2.], dtype=float32), 'action_mask': array([1, 1, 0, 1, 1], dtype=int8)}
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 1

Observation: {'observation': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 2.], dtype=float32), 'action_mask': array([1, 1, 0, 1, 1], dtype=int8)}
Reward: 0
Termination: False
Truncation: False
Return: 0
        
Action: 1

Observation: {'observation': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 